# Byte-Pair Encoding tokenization



Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [2]:
%%capture
!pip install datasets evaluate transformers[sentencepiece]

Also log into Hugging Face.

In [3]:
from huggingface_hub import notebook_login

notebook_login()

Byte-Pair Encoding (<font color='blue'>BPE</font>) was initially developed as an algorithm to <font color='blue'>compress texts</font>, and then used by <font color='blue'>OpenAI</font> for <font color='blue'>tokenization</font> when <font color='blue'>pretraining</font> the <font color='blue'>GPT</font> model. It's used by a lot of Transformer models, including GPT, GPT-2, RoBERTa, BART, and DeBERTa.

## Training algorithm

BPE training <font color='blue'>starts</font> by <font color='blue'>computing</font> the <font color='blue'>unique set</font> of <font color='blue'>words</font> used in the <font color='blue'>corpus</font> (after the normalization and pre-tokenization steps are completed), then <font color='blue'>building</font> the <font color='blue'>vocabulary</font> by taking all the <font color='blue'>symbols</font> used to <font color='blue'>write</font> those <font color='blue'>words</font>. As an example, let's say our corpus uses these five words:

```
"hug", "pug", "pun", "bun", "hugs"
```

The base vocabulary will then be `["b", "g", "h", "n", "p", "s", "u"]`. For <font color='blue'>real-world</font> cases, that <font color='blue'>base vocabulary</font> will contain all the <font color='blue'>ASCII</font> characters, at the very least, and probably some <font color='blue'>Unicode</font> characters as well. If an example you are tokenizing uses a <font color='blue'>character</font> that is <font color='blue'>not</font> in the <font color='blue'>training corpus</font>, that character will be <font color='blue'>converted</font> to the <font color='blue'>unknown token</font>. That's one reason why lots of NLP models are very bad at analyzing content with emojis, for instance.

<Tip>

💡The GPT-2 and RoBERTa tokenizers (which are pretty similar) have a clever way to deal with this: they <font color='blue'>don't look</font> at <font color='blue'>words</font> as being written with <font color='blue'>Unicode characters</font>, but with <font color='blue'>bytes</font>. This way the <font color='blue'>base vocabulary</font> has a <font color='blue'>small size</font> (256), but <font color='blue'>every character</font> you can think of will still be included and <font color='blue'>not</font> end up being <font color='blue'>converted</font> to the <font color='blue'>unknown token</font>. This trick is called <font color='blue'>byte-level BPE</font>.

</Tip>

After getting this <font color='blue'>base vocabulary</font>, we add new tokens until the desired vocabulary size is reached by learning <font color='blue'>merges</font>, which are <font color='blue'>rules</font> to <font color='blue'>merge two elements</font> of the existing vocabulary together into a new one. So, at the beginning these merges will <font color='blue'>create tokens</font> with <font color='blue'>two characters</font>, and then, as training progresses, longer subwords.

At any step during the tokenizer training, the BPE algorithm will search for the <font color='blue'>most frequent pair</font> of <font color='blue'>existing tokens</font> (by "pair," here we mean <font color='blue'>two consecutive tokens</font> in a word). That <font color='blue'>most frequent pair</font> is the one that will be <font color='blue'>merged</font>, and we rinse and repeat for the next step.

Going back to our previous example, let's assume the words had the following frequencies:

```
("hug", 10), ("pug", 5), ("pun", 12), ("bun", 4), ("hugs", 5)
```

meaning `"hug"` was present 10 times in the corpus, `"pug"` 5 times, `"pun"` 12 times, `"bun"` 4 times, and `"hugs"` 5 times. We start the training by <font color='blue'>splitting</font> each <font color='blue'>word</font> into <font color='blue'>characters</font> (the ones that form our initial vocabulary) so we can see each word as a list of tokens:

```
("h" "u" "g", 10), ("p" "u" "g", 5), ("p" "u" "n", 12), ("b" "u" "n", 4), ("h" "u" "g" "s", 5)
```

Then we look at pairs. The pair `("h", "u")` is present in the words `"hug"` and `"hugs"`, so 15 times total in the corpus. It's not the most frequent pair, though: that honor belongs to <font color='blue'>`("u", "g")`</font>, which is present in `"hug"`, `"pug"`, and `"hugs"`, for a grand total of <font color='blue'>20 times</font> in the vocabulary.

Thus, the <font color='blue'>first merge rule</font> learned by the <font color='blue'>tokenizer</font> is <font color='blue'>`("u", "g") -> "ug"`</font>, which means that `"ug"` will be <font color='blue'>added</font> to the <font color='blue'>vocabulary</font>, and the <font color='blue'>pair</font> should be <font color='blue'>merged</font> in <font color='blue'>all</font> the <font color='blue'>words</font> of the <font color='blue'>corpus</font>. At the end of this stage, the vocabulary and corpus look like this:

```
Vocabulary: ["b", "g", "h", "n", "p", "s", "u", "ug"]
Corpus: ("h" "ug", 10), ("p" "ug", 5), ("p" "u" "n", 12), ("b" "u" "n", 4), ("h" "ug" "s", 5)
```

Now we have some pairs that result in a <font color='blue'>token longer</font> than <font color='blue'>two characters</font>: the pair <font color='blue'>`("h", "ug")`</font>, for instance (present 15 times in the corpus). The most frequent pair at this stage is `("u", "n")`, however, present 16 times in the corpus, so the <font color='blue'>second merge rule</font> learned is <font color='blue'>`("u", "n") -> "un"`</font>. Adding that to the vocabulary and merging all existing occurrences leads us to:

```
Vocabulary: ["b", "g", "h", "n", "p", "s", "u", "ug", "un"]
Corpus: ("h" "ug", 10), ("p" "ug", 5), ("p" "un", 12), ("b" "un", 4), ("h" "ug" "s", 5)
```

Now the most frequent pair is <font color='blue'>`("h", "ug")`</font>, so we learn the merge rule <font color='blue'>`("h", "ug") -> "hug"`</font>, which gives us our first <font color='blue'>three-letter token</font>. After the merge, the corpus looks like this:

```
Vocabulary: ["b", "g", "h", "n", "p", "s", "u", "ug", "un", "hug"]
Corpus: ("hug", 10), ("p" "ug", 5), ("p" "un", 12), ("b" "un", 4), ("hug" "s", 5)
```

And we continue like this until we reach the desired vocabulary size.

<Tip>

✏️ **Now your turn!** What do you think the next merge rule will be?

</Tip>

The current state of the corpus after the previous merges is:

```
Vocabulary: ["b", "g", "h", "n", "p", "s", "u", "ug", "un"]
Corpus: ("h" "ug", 10), ("p" "ug", 5), ("p" "un", 12), ("b" "un", 4), ("h" "ug" "s", 5)
```

We need to identify all the <font color='blue'>pairs</font> and count their <font color='blue'>frequencies</font>:

*  ("p", "ug"): appears in "pug" (5 times) = 5 times
*  ("p", "un"): appears in "pun" (12 times) = 12 times
*  ("b", "un"): appears in "bun" (4 times) = 4 times
*  ("hug", "s"): appears in "hugs" (5 times) = 5 times

The <font color='blue'>most frequent pair</font> is <font color='blue'>("p", "un")</font> with <font color='blue'>12 occurrences</font>.
Therefore, the <font color='blue'>next merge rule</font> will be <font color='blue'>("p", "un") -> "pun"</font>. This will <font color='blue'>add "pun"</font> to the <font color='blue'>vocabulary</font> and <font color='blue'>merge</font> all occurrences of the pair <font color='blue'>("p", "un")</font> in the <font color='blue'>corpus</font>. After this merge, the current state of the corpus is:

```
Vocabulary: ["b", "g", "h", "n", "p", "s", "u", "ug", "un", "hug", "pun"]
Corpus: ("hug", 10), ("p" "ug", 5), ("pun", 12), ("b" "un", 4), ("hug" "s", 5)
```

## Implementing BPE

Now let's take a look at an <font color='blue'>implementation</font> of the <font color='blue'>BPE algorithm</font>. This won't be an <font color='blue'>optimized version</font> you can actually use on a <font color='blue'>big corpus</font>; we just want to show you the code so you can understand the algorithm a little bit better.

First we need a corpus, so let's create a simple one with a few sentences:

In [4]:
corpus = [
    "This is the Hugging Face Course.",
    "This chapter is about tokenization.",
    "This section shows several tokenizer algorithms.",
    "Hopefully, you will be able to understand how they are trained and generate tokens.",
]

Next, we need to <font color='blue'>pre-tokenize</font> that <font color='blue'>corpus</font> into <font color='blue'>words</font>. Since we are replicating a <font color='blue'>BPE tokenizer</font> (like GPT-2), we will use the <font color='blue'>`gpt2` tokenizer</font> for the pre-tokenization:

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")

Then we <font color='blue'>compute</font> the <font color='blue'>frequencies</font> of <font color='blue'>each word</font> in the corpus as we do the pre-tokenization:


In [7]:
from collections import defaultdict

word_freqs = defaultdict(int)

for text in corpus:
    words_with_offsets = tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)
    new_words = [word for word, offset in words_with_offsets]
    for word in new_words:
        word_freqs[word] += 1

# Display word frequencies
for i, (word, freq) in enumerate(word_freqs.items()):
    if i == len(word_freqs) - 1:
        print(f"    '{word}': {freq}")
    else:
        print(f"    '{word}': {freq},")

    'This': 3,
    'Ġis': 2,
    'Ġthe': 1,
    'ĠHugging': 1,
    'ĠFace': 1,
    'ĠCourse': 1,
    '.': 4,
    'Ġchapter': 1,
    'Ġabout': 1,
    'Ġtokenization': 1,
    'Ġsection': 1,
    'Ġshows': 1,
    'Ġseveral': 1,
    'Ġtokenizer': 1,
    'Ġalgorithms': 1,
    'Hopefully': 1,
    ',': 1,
    'Ġyou': 1,
    'Ġwill': 1,
    'Ġbe': 1,
    'Ġable': 1,
    'Ġto': 1,
    'Ġunderstand': 1,
    'Ġhow': 1,
    'Ġthey': 1,
    'Ġare': 1,
    'Ġtrained': 1,
    'Ġand': 1,
    'Ġgenerate': 1,
    'Ġtokens': 1


The next step is to <font color='blue'>compute</font> the <font color='blue'>base vocabulary</font>, formed by <font color='blue'>all the characters</font> used in the corpus:

In [8]:
alphabet = []

for word in word_freqs.keys():
    for letter in word:
        if letter not in alphabet:
            alphabet.append(letter)
alphabet.sort()

print(alphabet)

[',', '.', 'C', 'F', 'H', 'T', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'y', 'z', 'Ġ']


We also <font color='blue'>add</font> the <font color='blue'>special tokens</font> used by the model at the <font color='blue'>beginning</font> of that <font color='blue'>vocabulary</font>. In the case of GPT-2, the only special token is <font color='blue'>`"<|endoftext|>"`</font>:

In [9]:
vocab = ["<|endoftext|>"] + alphabet.copy()

We now need to <font color='blue'>split</font> each <font color='blue'>word</font> into <font color='blue'>individual characters</font>, to be able to start training:

In [10]:
splits = {word: [c for c in word] for word in word_freqs.keys()}

Now that we are ready for training, let's write a <font color='blue'>function</font> that <font color='blue'>computes</font> the <font color='blue'>frequency</font> of <font color='blue'>each pair</font>. We'll need to use this at each step of the training:

In [11]:
def compute_pair_freqs(splits):
    pair_freqs = defaultdict(int)
    for word, freq in word_freqs.items():
        split = splits[word]
        if len(split) == 1:
            continue
        for i in range(len(split) - 1):
            pair = (split[i], split[i + 1])
            pair_freqs[pair] += freq
    return pair_freqs

Let's have a <font color='blue'>look</font> at a <font color='blue'>part</font> of this <font color='blue'>dictionary</font> after the initial splits:

In [12]:
pair_freqs = compute_pair_freqs(splits)

for i, key in enumerate(pair_freqs.keys()):
    print(f"{key}: {pair_freqs[key]}")
    if i >= 5:
        break

('T', 'h'): 3
('h', 'i'): 3
('i', 's'): 5
('Ġ', 'i'): 2
('Ġ', 't'): 7
('t', 'h'): 3


Now, finding the <font color='blue'>most frequent pair</font> only takes a <font color='blue'>quick loop</font>:

In [13]:
best_pair = ""
max_freq = None

for pair, freq in pair_freqs.items():
    if max_freq is None or max_freq < freq:
        best_pair = pair
        max_freq = freq

print(best_pair, max_freq)

('Ġ', 't') 7


So the <font color='blue'>first merge</font> to learn is <font color='blue'>`('Ġ', 't') -> 'Ġt'`</font>, and we add <font color='blue'>`'Ġt'`</font> to the vocabulary:




In [14]:
merges = {("Ġ", "t"): "Ġt"}
vocab.append("Ġt")

To continue, we need to apply that <font color='blue'>merge</font> in our <font color='blue'>`splits` dictionary</font>. Let's write another function for this:

In [15]:
def merge_pair(a, b, splits):
    for word in word_freqs:
        split = splits[word]
        if len(split) == 1:
            continue

        i = 0
        while i < len(split) - 1:
            if split[i] == a and split[i + 1] == b:
                split = split[:i] + [a + b] + split[i + 2 :]
            else:
                i += 1
        splits[word] = split
    return splits


And we can have a <font color='blue'>look</font> at the <font color='blue'>result</font> of the <font color='blue'>first merge</font>:

In [16]:
splits = merge_pair("Ġ", "t", splits)
print(splits["Ġtrained"])

['Ġt', 'r', 'a', 'i', 'n', 'e', 'd']


Now we have everything we need to <font color='blue'>loop</font> until we have <font color='blue'>learned all</font> the <font color='blue'>merges</font> we want. Let's aim for a <font color='blue'>vocab size</font> of <font color='blue'>50</font>:

In [17]:
vocab_size = 50

while len(vocab) < vocab_size:
    pair_freqs = compute_pair_freqs(splits)
    best_pair = ""
    max_freq = None
    for pair, freq in pair_freqs.items():
        if max_freq is None or max_freq < freq:
            best_pair = pair
            max_freq = freq
    splits = merge_pair(*best_pair, splits)
    merges[best_pair] = best_pair[0] + best_pair[1]
    vocab.append(best_pair[0] + best_pair[1])

As a result, we've learned <font color='blue'>19 merge rules</font> (the initial vocabulary had a size of <font color='blue'>31 -- 30 characters</font> in the <font color='blue'>alphabet</font>, plus the <font color='blue'>special token</font>):

In [21]:
#print(merges)
for i, (word, freq) in enumerate(merges.items()):
    if i == len(merges) - 1:
        print(f"{i+1}:    '{word}': {freq}")
    else:
        print(f"{i+1}:     '{word}': {freq},")

1:     '('Ġ', 't')': Ġt,
2:     '('i', 's')': is,
3:     '('e', 'r')': er,
4:     '('Ġ', 'a')': Ġa,
5:     '('Ġt', 'o')': Ġto,
6:     '('e', 'n')': en,
7:     '('T', 'h')': Th,
8:     '('Th', 'is')': This,
9:     '('o', 'u')': ou,
10:     '('s', 'e')': se,
11:     '('Ġto', 'k')': Ġtok,
12:     '('Ġtok', 'en')': Ġtoken,
13:     '('n', 'd')': nd,
14:     '('Ġ', 'is')': Ġis,
15:     '('Ġt', 'h')': Ġth,
16:     '('Ġth', 'e')': Ġthe,
17:     '('i', 'n')': in,
18:     '('Ġa', 'b')': Ġab,
19:    '('Ġtoken', 'i')': Ġtokeni


And the <font color='blue'>vocabulary</font> is composed of the <font color='blue'>special token</font>, the <font color='blue'>initial alphabet</font>, and all the <font color='blue'>results</font> of the <font color='blue'>merges</font>:

In [24]:
for ele in vocab:
    print(ele)

<|endoftext|>
,
.
C
F
H
T
a
b
c
d
e
f
g
h
i
k
l
m
n
o
p
r
s
t
u
v
w
y
z
Ġ
Ġt
is
er
Ġa
Ġto
en
Th
This
ou
se
Ġtok
Ġtoken
nd
Ġis
Ġth
Ġthe
in
Ġab
Ġtokeni


<Tip>

💡 Using <font color='blue'>`train_new_from_iterator()`</font> on the <font color='blue'>same corpus won't result</font> in the exact <font color='blue'>same vocabulary</font>. This is because when there is a <font color='blue'>choice</font> of the <font color='blue'>most frequent pair</font>, <font color='blue'>we selected</font> the <font color='blue'>first one encountered</font>, while the 🤗 <font color='blue'>Tokenizers library</font> selects the <font color='blue'>first one</font> based on its <font color='blue'>inner IDs</font>.

</Tip>

To tokenize a new text, we pre-tokenize it, split it, then apply all the merge rules learned:

In [25]:
def tokenize(text):
    pre_tokenize_result = tokenizer._tokenizer.pre_tokenizer.pre_tokenize_str(text)
    pre_tokenized_text = [word for word, offset in pre_tokenize_result]
    splits = [[l for l in word] for word in pre_tokenized_text]
    for pair, merge in merges.items():
        for idx, split in enumerate(splits):
            i = 0
            while i < len(split) - 1:
                if split[i] == pair[0] and split[i + 1] == pair[1]:
                    split = split[:i] + [merge] + split[i + 2 :]
                else:
                    i += 1
            splits[idx] = split

    return sum(splits, [])

We can try this on <font color='blue'>any text</font> composed of <font color='blue'>characters</font> in the <font color='blue'>alphabet</font>:

In [26]:
tokenize("This is not a token.")

['This', 'Ġis', 'Ġ', 'n', 'o', 't', 'Ġa', 'Ġtoken', '.']

<Tip warning={true}>

⚠️ Our implementation will <font color='blue'>throw an error</font> if there is an <font color='blue'>unknown character</font> since we didn't do anything to handle them. <font color='blue'>GPT-2 doesn't</font> actually <font color='blue'>have</font> an <font color='blue'>unknown token</font> (it's <font color='blue'>impossible</font> to get an <font color='blue'>unknown character</font> when using <font color='blue'>byte-level BPE</font>), but this could happen here because we did not include <font color='blue'>all</font> the <font color='blue'>possible bytes</font> in the <font color='blue'>initial vocabulary</font>. This aspect of BPE is beyond the scope of this section, so we've left the details out.

</Tip>

That's it for the BPE algorithm! Next, we'll have a look at WordPiece.